# 🖼️ Week2: GPT-5 이미지 실습 (Colab)



> **주의:** Colab 런타임을 재시작하면 `OPENAI_API_KEY` 환경 변수를 다시 설정해야 합니다.



## 진행 순서
1. 환경 준비 및 기본 이미지 생성
2. 이미지 저장 · 표시 및 출력 옵션 실습
3. 멀티턴 이미지 개선(Responses API)
4. 이미지 분석(비전) 기본 실습
5. 선택 과제 & 확장 아이디어



## 1) 환경 준비 및 기본 이미지 생성
OpenAI 이미지 생성/분석을 위해 최신 SDK와 보조 라이브러리를 준비합니다.



In [ ]:
%pip install openai pillow



### API 키 설정
Colab 또는 로컬 환경에서 `OPENAI_API_KEY`를 환경 변수로 등록합니다.



In [ ]:
import os
from getpass import getpass

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ API 키 설정 완료")



In [ ]:
from openai import OpenAI

client = OpenAI()
print("✅ 클라이언트 생성 완료")



### GPT Image 빠르게 살펴보기
- `gpt-image-1` 모델은 텍스트·이미지를 모두 이해하고 생성할 수 있는 멀티모달 모델입니다.
- Responses API에서 `tools=[{"type": "image_generation"}]`을 사용하면 대화 흐름 안에 이미지 생성을 통합할 수 있습니다.
- 아래 실습에서는 Responses API를 활용한 이미지 생성 프로세스를 직접 구현해 봅니다.



In [ ]:
import base64



In [ ]:
def generate_image(prompt: str, *, size: str = "1024x1024", quality: str = "medium") -> bytes:
    """텍스트 프롬프트를 기반으로 이미지를 생성합니다.

    Args:
        prompt: 생성하고 싶은 이미지 설명.
        size: 출력 해상도 (예: "1024x1024", "1024x1536").
        quality: 이미지 품질 ("low", "medium", "high").
    Returns:
        base64 디코딩된 이미지 바이트.
    """
    # TODO 1) Responses API 호출 시 image_generation 도구를 등록하세요.
    # TODO 2) 응답 객체에서 image_generation_call 타입을 찾아 base64 결과를 추출하세요.
    # TODO 3) base64 문자열을 디코딩하여 bytes로 반환하세요.
    raise NotImplementedError("TODO: 이미지 생성 로직을 구현하세요.")



> 실습 팁: `prompt`뿐 아니라 `size`, `quality`를 바꿔 보고 결과가 어떻게 달라지는지 비교해 보세요.



In [ ]:
# TODO: 위 함수를 구현한 뒤 셀을 실행해 이미지를 생성하고 저장해 보세요.
# img_bytes = generate_image(
#     "A cozy reading nook with floor-to-ceiling bookshelves and warm lighting",
#     size="1024x1024",
#     quality="high",
# )
# with open("reading_nook.png", "wb") as f:
#     f.write(img_bytes)



## 2) 이미지 저장 · 표시 및 출력 옵션 실습
생성된 이미지를 즉시 확인하거나 Colab 노트북에 표시해 보세요.



In [ ]:
from IPython.display import display, Image as DisplayImage

def show_image(image_path: str) -> None:
    """저장된 이미지를 노트북에서 바로 확인합니다."""
    display(DisplayImage(filename=image_path))

# TODO: 이미지 생성 후 아래 함수를 호출하여 결과를 시각적으로 검토해 보세요.
# show_image("reading_nook.png")



> 참고: `size`, `quality`, `background` 등 추가 파라미터는 이미지 생성 도구 호출 시 함께 전달할 수 있습니다. (예: 투명 배경, 세로형 비율 등)



## 3) 멀티턴 이미지 개선 실습
`previous_response_id`를 활용하면 이전 이미지 생성을 이어받아 수정 지시를 내릴 수 있습니다.



In [ ]:
def refine_image(prompt: str, follow_up: str) -> tuple[str, bytes]:
    """하나의 프롬프트를 생성한 뒤 후속 지시로 이미지를 재생성합니다."""
    # TODO 1) 최초 생성: generate_image와 동일하게 Responses API를 호출하고 response.id를 보관하세요.
    # TODO 2) 후속 호출: previous_response_id를 사용해 follow_up 지시를 전달하세요.
    # TODO 3) 두 번째 응답에서 base64 이미지를 추출해 bytes로 반환하세요.
    raise NotImplementedError("TODO: 멀티턴 이미지 개선 로직을 구현하세요.")



In [ ]:
# TODO: 함수 구현 후 아래 예제 프롬프트를 활용해 전/후 이미지를 비교해 보세요.
# first_prompt = "Generate a stylized poster of a futuristic city skyline at sunset"
# follow_up_prompt = "Increase the level of realism and add flying cars"
# refined_id, refined_bytes = refine_image(first_prompt, follow_up_prompt)
# with open("city_refined.png", "wb") as f:
#     f.write(refined_bytes)
# show_image("city_refined.png")



> 참고: 후속 요청에서는 `tools=[{"type": "image_generation"}]` 구성을 유지하되 `previous_response_id` 또는 `image_generation_call`의 `id`를 활용해 히스토리를 이어갈 수 있습니다.



## 4) 이미지 분석(비전) 기본 실습
이미지를 입력으로 전달해 장면을 설명하거나 세부 정보를 추출하는 요청을 만들어 봅니다

In [ ]:
def describe_image(image_url: str, question: str, *, detail: str = "auto") -> str:
    """이미지 URL을 입력으로 받아 모델의 설명을 반환합니다."""
    # TODO 1) Responses API에 이미지 URL과 질문 텍스트를 함께 전달하세요.
    # TODO 2) 필요하다면 content 항목에 `{"detail": detail}` 값을 포함하세요.
    # TODO 3) 모델이 생성한 텍스트 설명을 반환하세요.
    raise NotImplementedError("TODO: 이미지 분석 로직을 구현하세요.")



In [ ]:
# TODO: 아래 이미지 URL을 활용해 장면 설명을 시도해 보세요.
# sample_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/1024px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
# print(describe_image(sample_url, "이미지 속 주요 대상과 분위기를 설명해 줘"))



> 확장: 로컬 이미지를 분석하려면 `base64.b64encode`로 인코딩하거나 Files API에 업로드한 뒤 `file_id`를 전달해 보세요.



## 5) 선택 과제 & 확장 아이디어
- 생성된 이미지를 Colab에서 바로 다운로드할 수 있도록 파일 링크를 표시해 보세요.
- `background="transparent"`, `output_format="webp"` 등 다양한 출력 옵션을 시도해 보세요.
- 하나의 대화 흐름에서 이미지에 설명을 추가하고, 그 결과를 텍스트로 정리해 보고서를 생성해 보세요.
- 스트리밍 옵션(`partial_images`)을 켜고 중간 생성 과정을 시각화해 보세요.



> 다음 단계: 실습을 마친 뒤에는 생성 이미지와 분석 결과를 공유하고, Week2의 비디오/TTS/STT 실습으로 이어가 보세요.

